Another source of potential information on species comes from the IUCN Red List. This notebook makes use of the iucn module and a species Red List search in the bispy package that requires an API key to operate against the IUCN API. It caches discovered information on these species for convenience in use.

# Data Management Considerations
The [IUCN Red List Terms and Conditions](https://www.iucnredlist.org/terms/terms-of-use) are fairly restrictive with regard to reposting, derivative works, and attribution. The API key that must be supplied by the user running our code for access to the IUCN Red List API is obtained by agreeing to adhere to the terms of use. We are currently exploring, with the IUCN team, the best ways that we can leverage these data, comply with the terms of use, and provide value back to the whole community. 

In [2]:
import requests
import bispy
from IPython.display import display
from joblib import Parallel, delayed
from collections import Counter

iucn = bispy.iucn.Iucn()
bis_utils = bispy.bis.Utils()

import helperfunctions

In [3]:
name_list = helperfunctions.workplan_species()

In [4]:
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
iucn_results = Parallel(n_jobs=8)(delayed(iucn.search_species)(name, name_source) for name, name_source in name_list)


In [5]:
iucn_results_filtered = [r for r in iucn_results if r["Processing Metadata"]["Status"] not in ["failure","Error"]]

In [6]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/iucn.json", iucn_results_filtered))

{'Doc Cache File': 'cache/iucn.json',
 'Document Number 184': {'IUCN Species Summary': {'Assessment Date': '2014-03-07',
   'Citation': 'Cordeiro, J. & Bogan, A.E. 2014. Fusconaia mitchelli. The IUCN Red List of Threatened Species 2014: e.T19056A1954826. http://dx.doi.org/10.2305/IUCN.UK.2014-3.RLTS.T19056A1954826.en .Downloaded on 10 July 2019',
   'Population Trend': 'Decreasing',
   'Report Link': 'http://dx.doi.org/10.2305/IUCN.UK.2014-3.RLTS.T19056A1954826.en',
   'Status': 'Critically Endangered',
   'Status Code': 'CR',
   'Taxon ID': 19056},
  'Processing Metadata': {'Date Processed': '2019-07-10T14:18:14.955401',
   'Name Source': 'Valid ITIS Scientific Name',
   'Scientific Name': 'Fusconaia mitchelli',
   'Search URL': 'http://apiv3.iucnredlist.org/api/v3/species/Fusconaia mitchelli',
   'Status': 'success',
   'Summary Result': 'Species Name Matched'}},
 'Number of Documents in Cache': 190}

# Interesting Facet
Perhaps the most interesting current use of the IUCN data is in the area of examining the IUCN Red List categories. The following code block reports on the number of work plan species not found in the IUCN system as a whole along with a listing of the numbers of species in each IUCN Red List category.

In [7]:
print("Not found in IUCN Red List:", len([r for r in iucn_results if r["Processing Metadata"]["Status"] == "Failure"]))

Counter(spp["IUCN Species Summary"]["Status"] for spp in iucn_results_filtered)

Not found in IUCN Red List: 0


Counter({'Critically Endangered': 8,
         'Data Deficient': 27,
         'Endangered': 32,
         'Least Concern': 17,
         'Near Threatened': 38,
         'Near Threatened (in review)': 10,
         'Vulnerable': 58})

In [9]:
Counter(spp["Processing Metadata"]["Name Source"] for spp in iucn_results_filtered)

Counter({'Lookup Name': 183, 'Valid ITIS Scientific Name': 7})